In [1]:
import os

import torch 
import torch.nn as nn
from torchvision.transforms import ToTensor, Compose

import numpy as np 
import gym
import importlib
import model.mdrnn  as mdrnn
MDRNN = importlib.reload(mdrnn).MDRNN

import dataset
dataset = importlib.reload(dataset)
RNNDataset = dataset.RNNDataset
MultiToTensor = dataset.MultiToTensor
from model.vaelin import VAELin
from constants import *

from torch.utils.data import DataLoader

In [2]:
from torchsummary import summary

In [14]:
batch_size=1
sequence_length=10
hidden_space_dim = 3 
action_space_dim=1
latent_space_dim=8 
num_mixtures=2
rnn_type="lstm"
n_layers=1
device=None


latents = torch.randn(batch_size, sequence_length , latent_space_dim)
next_latents = torch.randn(batch_size, sequence_length , latent_space_dim)
actions = torch.randn(batch_size ,sequence_length, action_space_dim)

In [15]:
model = MDRNN(batch_size, sequence_length, hidden_space_dim, action_space_dim, latent_space_dim, num_mixtures, rnn_type, n_layers, device)

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
vae = VAELin(z_size=16, device=device).to(device)
loaded = torch.load(VAE_PATH)
vae.load_state_dict(loaded['model_state_dict'])
#%%
dataset = RNNDataset(transform=MultiToTensor())
batch_size = 10
loader = DataLoader(dataset, batch_size=batch_size)

#%%
def to_latent(batch_frames, vae):
    frames = batch_frames.reshape((batch_size, RNN_SAMPLE_SIZE, 3, HEIGHT, WIDTH))
    latent_seq = vae(frames)[3]
    latent_seqs = latent_seq.reshape((batch_size, RNN_SAMPLE_SIZE, 16))
    return latent_seqs

#%%
for batch in loader:
    frame_seqs = batch['frame_seq'] # (BATCHSIZE, SEQ_LEN, 3, 64, 64)
    action_seqs = batch['action_seq'] # (BATCHSIZE, SEQ_LEN)
    latent_seqs = to_latent(frame_seqs, vae) # (BATCHSIZE, SEQ_LEN, LATENT_SIZE)

In [17]:
action_seqs

torch.Size([10, 100])

In [ ]:
t1 t2 t3
(l, a) --> (n_l)